# Neural Network Bias Variability Demonstration

This notebook demonstrates how bias in a simple neural network model can vary with each retraining when using a dynamically set seed. We'll use the Iris dataset for the demonstration.

In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## Step 1: Load and Preprocess Data

In [ ]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

## Step 2: Define a Function to Set a Dynamic Seed

In [ ]:
def set_dynamic_seed():
    """Set a random seed using system entropy for bias variability."""
    seed = int.from_bytes(os.urandom(4), "big")
    np.random.seed(seed)
    tf.random.set_seed(seed)
    return seed

## Step 3: Create a Simple Neural Network Model

In [ ]:
def create_model():
    """Define a neural network model."""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Step 4: Train the Model Multiple Times and Record Biases

In [ ]:
bias_variations = []

for i in range(5):  # Train the model 5 times
    current_seed = set_dynamic_seed()
    model = create_model()
    model.fit(X_train, y_train, epochs=5, verbose=0)  # Short training for demonstration
    bias = model.layers[1].get_weights()[1]  # Extract bias from the second layer
    bias_variations.append((current_seed, bias))
    print(f"Run {i + 1} | Seed: {current_seed} | Bias: {bias}")

## Step 5: Display the Recorded Biases

In [ ]:
for i, (seed, bias) in enumerate(bias_variations):
    print(f"Run {i + 1}: Seed = {seed}")
    print(f"Biases: {bias}")
    print("-")